# Experiment result:

|  | 0 | 1/9 | 4/9 | 1/2 | 1 |
|-------------------|--------|--------|---------|-----|-------|
| pre-trained model | 0.7275 | 0.6069 | 0.26025 |  | 0.041 |
| fine tuned on 0 |  |  |  |  |  |
| fine tuned on all |  |  |  |  |  |

In [ ]:
# Choose classes.
with open("../data/imagenet/ILSVRC2015_CLS_LOC_synset_list.txt") as f:
    content = f.readlines()
len(content)
from random import randint
r = [randint(0, 1002) for x in range(20)]
print r
l = [content[i] for i in r]
for x in l:
    print x

In [5]:
# Generate training dataset and test dataset.
import os
import numpy as np
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
import copy
from matplotlib.pyplot import imshow
%matplotlib inline

imagenet_root = '/home/haow3/occlusion-project/data/imagenet/'
caffe_root = '/home/haow3/software/caffe-rc3/'

# Load labels.
imagenet_labels_filename = caffe_root + 'data/ilsvrc12/synset_words.txt'
label_to_wnid = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')
wnid_to_label = {}
for label in range(len(label_to_wnid)):
    wnid = label_to_wnid[label].split(' ')[0]
    wnid_to_label[wnid] = label

synset_names = os.listdir(imagenet_root + 'image/')

training_dataset = [(0.0, 0), (1.0/3, 3), (1.0/2, 3), (2.0/3, 2), (1.0, 1)]
test_dataset = [(0.0, 0), (1.0/3, 3), (1.0/2, 3), (2.0/3, 2), (1.0, 1)]

train_files = []
test_files = []

for (occlu_size, occlu_num) in training_dataset:
    percent = str(int(100 * occlu_size * occlu_size))
    print percent
    f = open('{}dataset/train_{}_{}.txt'.format(imagenet_root, percent, str(occlu_num)), 'w')
    train_files.append(f)
    if not os.path.exists('{}dataset/train_{}_{}'.format(imagenet_root, percent, str(occlu_num))):
        os.makedirs('{}dataset/train_{}_{}'.format(imagenet_root, percent, str(occlu_num)))
        

for (occlu_size, occlu_num) in test_dataset:
    percent = str(int(100 * occlu_size * occlu_size))
    print percent
    f = open('{}dataset/test_{}_{}.txt'.format(imagenet_root, percent, str(occlu_num)), 'w')
    test_files.append(f)
    if not os.path.exists('{}dataset/test_{}_{}'.format(imagenet_root, percent, str(occlu_num))):
        os.makedirs('{}dataset/test_{}_{}'.format(imagenet_root, percent, str(occlu_num)))

# occluder size = occlu_size * occlu_size
# occluder num = occlu_num * occlu_num
# path = 'imagenet_root/dataset/train_0/wnid'
def generate_datum(img_orig, path, f, class_id, rects, occlu_size, occlu_num):
    if occlu_size == 0:
        datum_path = '{}_0.jpeg'.format(path)
        img_orig.save(datum_path)
        f.write('{} {}\n'.format(datum_path, str(class_id)))
        return
    for i in range(occlu_num):
        for j in range(occlu_num):
            img = copy.copy(img_orig)
            d = ImageDraw.Draw(img)
            if (occlu_num == 1):
                delta = 1
            else:
                delta = (1 - occlu_size) / float(occlu_num - 1)
            for rect in rects:
                subrect = [0, 0, 0, 0]
                subrect[0] = rect[0] + i * (rect[2] - rect[0]) * delta
                subrect[1] = rect[1] + j * (rect[3] - rect[1]) * delta
                subrect[2] = rect[0] + (i + 1) * (rect[2] - rect[0]) * delta
                subrect[3] = rect[1] + (j + 1) * (rect[3] - rect[1]) * delta
                d.rectangle(subrect, fill="black", outline=None)
            datum_path = '{}_{}_{}_{}.jpeg'.format(path, str(int(100 * occlu_size * occlu_size)), str(i), str(j))
            img.save(datum_path)
            f.write('{} {}\n'.format(datum_path, str(class_id)))
             
for synset_name in synset_names:
    image_names = os.listdir(imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images')
    annotation_names = os.listdir(imagenet_root + 'Annotation/' + synset_name + '/')
    n1 = [os.path.splitext(n)[0] for n in image_names]
    n2 = [os.path.splitext(n)[0] for n in annotation_names]
    intersection_names = list(set(n1) & set(n2))
    # train : test = 300 : 100
    for i in range(400):
        print 'Processing: ', intersection_names[i]
        # Read bounding box.
        bbx_file = open(imagenet_root + 'Annotation/' + synset_name + '/' + intersection_names[i] + '.xml')
        xmltree = ET.parse(bbx_file)
        objects = xmltree.findall('object')
        rects = []
        for obj in objects:
            bbx = obj.find('bndbox')
            rects.append([int(it.text) for it in bbx])
            
        img_orig = Image.open(imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images/' + intersection_names[i] + '.JPEG')
        if i < 300: # Training dataset. 
            continue
            # 0%
            img = copy.copy(img_orig)
            filepath = '{}dataset/train_0/{}_0.jpeg'.format(imagenet_root, intersection_names[i])
            img.save(filepath)
            train_0_file.write('{} {}\n'.format(filepath, str(wnid_to_label[synset_name])))
            
            filepath = '{}dataset/train_0_11_44/{}_0.jpeg'.format(imagenet_root, intersection_names[i])
            img.save(filepath)
            train_0_11_44_file.write('{} {}\n'.format(filepath, str(wnid_to_label[synset_name])))
            
            # 11% = 1/9
            for ri in range(3):
                for rj in range(3):
                    img = copy.copy(img_orig)
                    d = ImageDraw.Draw(img)
                    for rect in rects:
                        subrect = [0, 0, 0, 0]
                        subrect[0] = rect[0] + ri * (rect[2] - rect[0]) / 3
                        subrect[1] = rect[1] + rj * (rect[3] - rect[1]) / 3
                        subrect[2] = rect[0] + (ri + 1) * (rect[2] - rect[0]) / 3
                        subrect[3] = rect[1] + (rj + 1) * (rect[3] - rect[1]) / 3
                        d.rectangle(subrect, fill="black", outline=None)
                    filepath = '{}dataset/train_0_11_44/{}_11_{}_{}.jpeg'.format(imagenet_root, intersection_names[i], ri, rj)
                    img.save(filepath)
                    train_0_11_44_file.write('{} {}\n'.format(filepath, str(wnid_to_label[synset_name])))
                        
            # 44% = 4/9
            for ri in range(2):
                for rj in range(2):
                    img = copy.copy(img_orig)
                    d = ImageDraw.Draw(img)
                    for rect in rects:
                        subrect = [0, 0, 0, 0]
                        subrect[0] = rect[0] + ri * (rect[2] - rect[0]) / 3
                        subrect[1] = rect[1] + rj * (rect[3] - rect[1]) / 3
                        subrect[2] = rect[0] + (ri + 2) * (rect[2] - rect[0]) / 3
                        subrect[3] = rect[1] + (rj + 2) * (rect[3] - rect[1]) / 3
                        d.rectangle(subrect, fill="black", outline=None)
                    filepath = '{}dataset/train_0_11_44/{}_44_{}_{}.jpeg'.format(imagenet_root, intersection_names[i], ri, rj)
                    img.save(filepath)
                    train_0_11_44_file.write('{} {}\n'.format(filepath, str(wnid_to_label[synset_name])))
            
        else: # Testing dataset.
            # 0%
            img = copy.copy(img_orig)
            filepath = '{}dataset/test_0/{}_0.jpeg'.format(imagenet_root, intersection_names[i])
            img.save(filepath)
            test_0_file.write('{} {}\n'.format(filepath, str(wnid_to_label[synset_name])))
            
            # 11% = 1/9
            for ri in range(3):
                for rj in range(3):
                    img = copy.copy(img_orig)
                    d = ImageDraw.Draw(img)
                    for rect in rects:
                        subrect = [0, 0, 0, 0]
                        subrect[0] = rect[0] + ri * (rect[2] - rect[0]) / 3
                        subrect[1] = rect[1] + rj * (rect[3] - rect[1]) / 3
                        subrect[2] = rect[0] + (ri + 1) * (rect[2] - rect[0]) / 3
                        subrect[3] = rect[1] + (rj + 1) * (rect[3] - rect[1]) / 3
                        d.rectangle(subrect, fill="black", outline=None)
                    filepath = '{}dataset/test_11/{}_11_{}_{}.jpeg'.format(imagenet_root, intersection_names[i], ri, rj)
                    img.save(filepath)
                    test_11_file.write('{} {}\n'.format(filepath, str(wnid_to_label[synset_name])))
            
            # 44% = 4/9
            for ri in range(2):
                for rj in range(2):
                    img = copy.copy(img_orig)
                    d = ImageDraw.Draw(img)
                    for rect in rects:
                        subrect = [0, 0, 0, 0]
                        subrect[0] = rect[0] + ri * (rect[2] - rect[0]) / 3
                        subrect[1] = rect[1] + rj * (rect[3] - rect[1]) / 3
                        subrect[2] = rect[0] + (ri + 2) * (rect[2] - rect[0]) / 3
                        subrect[3] = rect[1] + (rj + 2) * (rect[3] - rect[1]) / 3
                        d.rectangle(subrect, fill="black", outline=None)
                    filepath = '{}dataset/test_44/{}_44_{}_{}.jpeg'.format(imagenet_root, intersection_names[i], ri, rj)
                    img.save(filepath)
                    test_44_file.write('{} {}\n'.format(filepath, str(wnid_to_label[synset_name])))
            
            # 100%
            img = copy.copy(img_orig)
            d = ImageDraw.Draw(img)
            for rect in rects:
                d.rectangle(rect, fill="black", outline=None)
            filepath = '{}dataset/test_100/{}_100.jpeg'.format(imagenet_root, intersection_names[i])
            img.save(filepath)
            test_100_file.write('{} {}\n'.format(filepath, str(wnid_to_label[synset_name])))
            
for f in train_files:
    f.close()
for f in test_files:
    f.close()
            


0
11
25
44
100
0
11
25
44
100


'\nfor synset_name in synset_names:\n    image_names = os.listdir(imagenet_root + \'image/\' + synset_name + \'/\' + synset_name + \'_original_images\')\n    annotation_names = os.listdir(imagenet_root + \'Annotation/\' + synset_name + \'/\')\n    n1 = [os.path.splitext(n)[0] for n in image_names]\n    n2 = [os.path.splitext(n)[0] for n in annotation_names]\n    intersection_names = list(set(n1) & set(n2))\n    # train : test = 300 : 100\n    for i in range(400):\n        print \'Processing: \', intersection_names[i]\n        # Read bounding box.\n        bbx_file = open(imagenet_root + \'Annotation/\' + synset_name + \'/\' + intersection_names[i] + \'.xml\')\n        xmltree = ET.parse(bbx_file)\n        objects = xmltree.findall(\'object\')\n        rects = []\n        for obj in objects:\n            bbx = obj.find(\'bndbox\')\n            rects.append([int(it.text) for it in bbx])\n            \n        img_orig = Image.open(imagenet_root + \'image/\' + synset_name + \'/\' + synset

In [ ]:
# Test.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import caffe
import sys

imagenet_root = '/home/haow3/occlusion-project/data/imagenet/'
caffe_root = '/home/haow3/software/caffe-rc3/'

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

caffe.set_mode_gpu()
net = caffe.Net(caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt',
                caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

# set net to batch size of 50
net.blobs['data'].reshape(50,3,227,227)

test_file = open(imagenet_root + 'dataset/test_44.txt', 'r')
positive_true = 0
lines = test_file.readlines()
image_sum = len(lines)
for i, line in enumerate(lines):
    # A possible optimization: add occluders on the test image before feeding them
    image_path, class_id = line.split(' ')
    class_id = int(class_id)
    net.blobs['data'].data[...] = transformer.preprocess('data', caffe.io.load_image(image_path))
    out = net.forward()
    predict_id = out['prob'][0].argmax()
    print '{}/{}: {} {} {}'.format(i, image_sum, image_path, predict_id, class_id)
    if class_id == predict_id:
        positive_true += 1
    
accuracy = positive_true / float(image_sum)
print accuracy


In [ ]:
# Fine tune.

import caffe
import numpy as np
from pylab import *
%matplotlib inline

# TODO